<a href="https://colab.research.google.com/github/koichiromasa/scsc-interface-kinetics/blob/main/frame_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Google Drive のマウント

In [ ]:
import cv2
import os
from google.colab import drive
from tqdm import tqdm

# Google Drive をマウント
drive.mount('/content/drive')


Mounted at /content/drive


2. 設定（入力・出力フォルダ

In [ ]:
# === 設定（入力・出力フォルダ） ===
input_video_path = "/content/drive/My Drive/Colab Notebooks/raw_data/POM_movie/25_120.mp4"
output_base_dir = "/content/drive/My Drive/Colab Notebooks/result_data/POM_movie"
video_name = os.path.splitext(os.path.basename(input_video_path))[0]

output_frames_dir = os.path.join(output_base_dir, video_name)
os.makedirs(output_frames_dir, exist_ok=True)

print(f"✅ 入力動画パス: {input_video_path}")
print(f"✅ 出力フォルダ: {output_frames_dir}")


✅ 入力動画パス: /content/drive/My Drive/Colab Notebooks/raw_data/POM_movie/25_120.mp4
✅ 出力フォルダ: /content/drive/My Drive/Colab Notebooks/result_data/POM_movie/25_120


3. 解析条件

In [ ]:
# 解析条件
start_temp = 120.0   # 開始温度 (°C)
heating_rate = 0   # 昇温速度 (°C/min)


4.  動画の情報を取得

In [ ]:
# === 動画の情報を取得 ===
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

if not cap.isOpened():
    print(f"⚠ エラー: 動画 {input_video_path} を開けませんでした。")
    exit()

print(f"🎥 動画情報: {frame_count} フレーム, {fps:.4f} FPS")


🎥 動画情報: 17433 フレーム, 19.3592 FPS


5.  動画をフレームごとに分割 & 保存

In [ ]:
# === フレーム分割 & 保存 ===
progress_bar = tqdm(total=frame_count, desc="Processing Frames", unit="frame")

for frame_idx in range(frame_count):
    ret, frame = cap.read()
    if not ret:
        break

    # === 時間 & 温度計算 ===
    time_sec = frame_idx / fps
    temp_c = start_temp + (heating_rate / 60.0) * time_sec

    # === ファイル名の設定 ===
    frame_filename = os.path.join(output_frames_dir, f"{frame_idx:06d}_{time_sec:.4f}s_{temp_c:.1f}C.png")

    # フレームを保存
    cv2.imwrite(frame_filename, frame)

    # 進行状況を更新
    progress_bar.update(1)

cap.release()
progress_bar.close()
print(f"\n✅ フレーム画像を {output_frames_dir} に保存しました")


Processing Frames: 100%|██████████| 17433/17433 [26:35<00:00, 10.92frame/s]


✅ フレーム画像を /content/drive/My Drive/Colab Notebooks/result_data/POM_movie/25_120 に保存しました


4. フレーム情報の保存

In [ ]:
# === フレーム情報を保存 ===
frame_info_path = os.path.join(output_frames_dir, "frame_info.txt")
with open(frame_info_path, "w") as f:
    f.write(f"Total Frames: {frame_count}\n")
    f.write(f"FPS: {fps:.4f}\n")
print(f"✅ フレーム情報を {frame_info_path} に保存しました")


✅ フレーム情報を /content/drive/My Drive/Colab Notebooks/result_data/POM_movie/25_120/frame_info.txt に保存しました
